In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings
import opendatasets as od
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
dataset = 'https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri'
od.download(dataset)
force=True

Skipping, found downloaded files in "./brain-tumor-classification-mri" (use force=True to force download)


In [3]:
for dirname, _, filenames in os.walk('./brain-tumor-classification-mri'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./brain-tumor-classification-mri/Training/Grade_2/m1(69).jpg
./brain-tumor-classification-mri/Training/Grade_2/m3 (126).jpg
./brain-tumor-classification-mri/Training/Grade_2/m3 (62).jpg
./brain-tumor-classification-mri/Training/Grade_2/m2 (62).jpg
./brain-tumor-classification-mri/Training/Grade_2/m1(86).jpg
./brain-tumor-classification-mri/Training/Grade_2/m (84).jpg
./brain-tumor-classification-mri/Training/Grade_2/m1(162).jpg
./brain-tumor-classification-mri/Training/Grade_2/m3 (171).jpg
./brain-tumor-classification-mri/Training/Grade_2/m2 (35).jpg
./brain-tumor-classification-mri/Training/Grade_2/m3 (35).jpg
./brain-tumor-classification-mri/Training/Grade_2/m1(135).jpg
./brain-tumor-classification-mri/Training/Grade_2/m1(28).jpg
./brain-tumor-classification-mri/Training/Grade_2/m1(6).jpg
./brain-tumor-classification-mri/Training/Grade_2/m3 (167).jpg
./brain-tumor-classification-mri/Training/Grade_2/m3 (23).jpg
./brain-tumor-classification-mri/Training/Grade_2/m3 (188).jpg
./brain-tu

./brain-tumor-classification-mri/Training/Grade_1/gg (362).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (698).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (732).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (227).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (677).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (765).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (335).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (620).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (270).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (159).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (53).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (509).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (323).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (773).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (289).jpg
./brain-tumor-classification-mri/Training/Grade_1/gg (26

In [4]:
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']
colors_red = ["#331313", "#582626", '#9E1717', '#D35151', '#E9B4B4']
colors_green = ['#01411C','#4B6F44','#4F7942','#74C365','#D0F0C0']

In [5]:
labels = ['Grade_1','Grade_2','Grade_3']

In [6]:
X_train = []
y_train = []
image_size = 150
for i in labels:
    folderPath = os.path.join(r'./brain-tumor-classification-mri','Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
        
for i in labels:
    folderPath = os.path.join(r'./brain-tumor-classification-mri','Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)
        
X_train = np.array(X_train)
y_train = np.array(y_train)

100%|███████████████████████████████████████| 105/105 [00:00<00:00, 1166.94it/s]


In [7]:
X_train, y_train = shuffle(X_train,y_train, random_state=101)

In [8]:
X_train.shape

(2363, 150, 150, 3)

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.1,random_state=101)

In [10]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [11]:
effnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))

In [12]:
model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(3,activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 150, 150, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 150, 150, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 150, 150, 3)  0           ['normalization[0][0]']      

 block2b_expand_conv (Conv2D)   (None, 38, 38, 144)  3456        ['block2a_project_bn[0][0]']     
                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 38, 38, 144)  576        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block2b_expand_activation (Act  (None, 38, 38, 144)  0          ['block2b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block2b_dwconv (DepthwiseConv2  (None, 38, 38, 144)  1296       ['block2b_expand_activation[0][0]
 D)                                                              ']                               
          

                                                                                                  
 block3b_dwconv (DepthwiseConv2  (None, 19, 19, 240)  6000       ['block3b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3b_bn (BatchNormalization  (None, 19, 19, 240)  960        ['block3b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3b_activation (Activation  (None, 19, 19, 240)  0          ['block3b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3b_s

 block4b_activation (Activation  (None, 10, 10, 480)  0          ['block4b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block4b_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block4b_se_squeeze[0][0]']     
                                                                                                  
 block4b_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block4b_se_reshape[0][0]']     
                                                                                                  
 block4b_s

                                                                                                  
 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
                                                                                                  
 block5a_se_expand (Conv2D)     (None, 1, 1, 480)    10080       ['block5a_se_reduce[0][0]']      
                                                                                                  
 block5a_se_excite (Multiply)   (None, 10, 10, 480)  0           ['block5a_activation[0][0]',     
                                                                  'block5a_se_expand[0][0]']      
                                                                                                  
 block5a_p

 block5c_project_conv (Conv2D)  (None, 10, 10, 112)  75264       ['block5c_se_excite[0][0]']      
                                                                                                  
 block5c_project_bn (BatchNorma  (None, 10, 10, 112)  448        ['block5c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block5c_drop (Dropout)         (None, 10, 10, 112)  0           ['block5c_project_bn[0][0]']     
                                                                                                  
 block5c_add (Add)              (None, 10, 10, 112)  0           ['block5c_drop[0][0]',           
                                                                  'block5b_add[0][0]']            
                                                                                                  
 block6a_e

 block6b_add (Add)              (None, 5, 5, 192)    0           ['block6b_drop[0][0]',           
                                                                  'block6a_project_bn[0][0]']     
                                                                                                  
 block6c_expand_conv (Conv2D)   (None, 5, 5, 1152)   221184      ['block6b_add[0][0]']            
                                                                                                  
 block6c_expand_bn (BatchNormal  (None, 5, 5, 1152)  4608        ['block6c_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block6c_expand_activation (Act  (None, 5, 5, 1152)  0           ['block6c_expand_bn[0][0]']      
 ivation)                                                                                         
          

 block7a_expand_bn (BatchNormal  (None, 5, 5, 1152)  4608        ['block7a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 5, 5, 1152)  0           ['block7a_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block7a_dwconv (DepthwiseConv2  (None, 5, 5, 1152)  10368       ['block7a_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block7a_bn (BatchNormalization  (None, 5, 5, 1152)  4608        ['block7a_dwconv[0][0]']         
 )        

In [14]:
model.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

In [15]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("effnet.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [16]:
history = model.fit(X_train,y_train,validation_split=0.1, epochs =12, verbose=1, batch_size=32,
                   callbacks=[tensorboard,checkpoint,reduce_lr])

Epoch 1/12
60/60 [==============================] - ETA: 0s - loss: 0.4588 - accuracy: 0.8160  
Epoch 1: val_accuracy improved from -inf to 0.79812, saving model to effnet.h5
60/60 [==============================] - 4321s 73s/step - loss: 0.4588 - accuracy: 0.8160 - val_loss: 1.1460 - val_accuracy: 0.7981 - lr: 0.0010
Epoch 2/12
60/60 [==============================] - ETA: 0s - loss: 0.1550 - accuracy: 0.9441
Epoch 2: val_accuracy improved from 0.79812 to 0.80751, saving model to effnet.h5
60/60 [==============================] - 198s 3s/step - loss: 0.1550 - accuracy: 0.9441 - val_loss: 0.7498 - val_accuracy: 0.8075 - lr: 0.0010
Epoch 3/12
60/60 [==============================] - ETA: 0s - loss: 0.1086 - accuracy: 0.9582
Epoch 3: val_accuracy improved from 0.80751 to 0.92019, saving model to effnet.h5
60/60 [==============================] - 208s 3s/step - loss: 0.1086 - accuracy: 0.9582 - val_loss: 0.3804 - val_accuracy: 0.9202 - lr: 0.0010
Epoch 4/12
60/60 [========================

In [17]:
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

8/8 [==============================] - 4s 349ms/step


In [18]:
print(classification_report(y_test_new,pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       107
           1       0.98      0.98      0.98        81
           2       1.00      1.00      1.00        49

    accuracy                           0.98       237
   macro avg       0.99      0.99      0.99       237
weighted avg       0.98      0.98      0.98       237



In [19]:
def img_pred(upload):
    for name, file_info in uploader.value.items():
        img = Image.open(io.BytesIO(file_info['content']))
    opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    img = cv2.resize(opencvImage,(150,150))
    img = img.reshape(1,150,150,3)
    p = model.predict(img)
    p = np.argmax(p,axis=1)[0]

    if p==0:
        p='Grade 1'
    elif p==1:
        p=='Grade 2'
    elif p==2:
        p='Grade 3'

    print(f'The Model predicts that it is a {p}')